In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark.conf.set(
    "fs.azure.account.key.stmeteoend2end.dfs.core.windows.net",
    "TU_KEY"
)

SILVER_PATH = "abfss://datalake@stmeteoend2end.dfs.core.windows.net/silver/weather"
GOLD_PATH   = "abfss://datalake@stmeteoend2end.dfs.core.windows.net/gold/weather"


In [0]:
df_silver = spark.read.format("delta").load(SILVER_PATH)
df_silver = df_silver.withColumn("date", to_date("timestamp"))

In [0]:
df_gold_hourly = (
    df_silver
    .groupBy(
        window("timestamp", "1 hour").alias("hour_window"),
        "ip"
    )
    .agg(
        avg("temperature").alias("avg_temperature"),
        min("temperature").alias("min_temperature"),
        max("temperature").alias("max_temperature"),
        avg("humidity").alias("avg_humidity"),
        avg("pm25").alias("avg_pm25"),
        max("uv_level").alias("max_uv"),
        avg("pressure").alias("avg_pressure"),
        avg("wind_raw").alias("avg_wind"),
        count("*").alias("events")
    )
    .withColumn("hour", col("hour_window").start)
    .drop("hour_window")
)

In [0]:
df_gold_daily = (
    df_silver
    .groupBy("date", "ip")
    .agg(
        avg("temperature").alias("avg_temperature"),
        max("uv_level").alias("max_uv"),
        avg("humidity").alias("avg_humidity"),
        avg("pm25").alias("avg_pm25"),
        avg("wind_raw").alias("avg_wind"),
        count("*").alias("events")
    )
)

In [0]:
df_gold = df_gold_hourly.withColumn("level", lit("hourly")).unionByName(
        df_gold_daily.withColumn("level", lit("daily")),
        allowMissingColumns=True
    )

In [0]:
df_gold.write.format("delta").mode("overwrite").save(GOLD_PATH)